# Executive Summary

This notebook aims to compile resources on route optimization. For aboitiz, this can be used in its delivery logistics such as in Basco and Republic Cement.

https://developers.google.com/optimization/routing

A more general version of the TSP is the vehicle routing problem (VRP), in which there are multiple vehicles. In most cases, VRPs have constraints: for example, vehicles might have capacities for the maximum weight or volume of items they can carry, or drivers might be required to visit locations during specified time windows requested by customers.

OR-Tools can solve many types of VRPs, including the following:

- Traveling Salesperson Problem, the classic routing problem in which there is just one vehicle.
- Vehicle routing problem, a generalisation of the TSP with multiple vehicles.
- VRP with capacity constraints, in which vehicles have maximum capacities for the items they can carry.
- VRP with time windows, where the vehicles must visit the locations in specified time intervals.
- VRP with resource constraints, such as space or personnel to load and unload vehicles at the depot (the starting point for the routes).
- VRP with dropped visits, where the vehicles aren't required to visit all locations, but must pay a penalty for each visit that is dropped.

Resources:
https://developers.google.com/optimization/routing/vrptw

# Loading Libraries

In [2]:
import os
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import pandas as pd

import plotly
import plotly.express as px
import plotly.graph_objects as go

import geopandas as gpd
import numpy as np

#import token
from mapbox_token import *

# Traveling Salesman Problem

The data also includes:

- The number of vehicles in the problem, which is 1 because this is a TSP. (For a vehicle routing problem (VRP), the number of vehicles can be greater than 1.)
- The depot: the start and end location for the route. In this case, the depot is 0, which corresponds to New York.


## Read data

In [3]:
os.chdir('data')
os.listdir()

['ESTABLISHMENTS_752020.csv']

In [4]:
df = pd.read_csv('ESTABLISHMENTS_752020.csv')

In [5]:
df.Company.value_counts().head(20)

BDO                           3015
711                           2709
BPI                           2374
BanKo                         2247
RCB                           1912
MBT                           1767
LBP                           1360
PNB                           1206
MERCURY DRUG                  1189
Cebuana Lhuillier Pawnshop    1179
JFC                           1079
SECB                          1069
CHIB                           931
EW                             908
Western Union                  896
Lbc                            863
Cebuana Lhuillier              757
Chooks To Go                   724
UBP                            713
Shell Gasoline Station         497
Name: Company, dtype: int64

In [6]:
#filter to one restautant and city first
jabee = df[(df.Company == 'JFC') & (df.MUNICIPALITY == 'Mandaluyong')].copy()
jabee = gpd.GeoDataFrame(jabee, geometry=gpd.points_from_xy(jabee.Longitude, jabee.Latitude))
jabee.reset_index(inplace=True)

In [7]:
jabee = jabee.set_crs('epsg:4326')
jabee = jabee.to_crs('epsg:3857')

In [8]:
dist = jabee.geometry.apply(lambda g: jabee.distance(g))/1000 #in km
dist

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.000000,1.467292,1.446216,2.801033,0.638357,0.227502,1.772084,3.232857,1.796228,2.066745,...,1.467292,1.446216,2.801033,0.638357,0.227502,1.772084,3.232857,1.796228,2.066745,1.270094
1,1.467292,0.000000,2.316423,1.388082,0.831212,1.648293,2.620063,1.924472,2.641841,1.072974,...,0.000000,2.316423,1.388082,0.831212,1.648293,2.620063,1.924472,2.641841,1.072974,1.928486
2,1.446216,2.316423,0.000000,3.250192,1.694945,1.266021,0.331978,3.426276,0.356196,2.210810,...,2.316423,0.000000,3.250192,1.694945,1.266021,0.331978,3.426276,0.356196,2.210810,0.417244
3,2.801033,1.388082,3.250192,0.000000,2.169268,2.950130,3.484957,0.654073,3.501550,1.073228,...,1.388082,3.250192,0.000000,2.169268,2.950130,3.484957,0.654073,3.501550,1.073228,2.833227
4,0.638357,0.831212,1.694945,2.169268,0.000000,0.818413,2.023159,2.627480,2.046917,1.518779,...,0.831212,1.694945,2.169268,0.000000,0.818413,2.023159,2.627480,2.046917,1.518779,1.377417
5,0.227502,1.648293,1.266021,2.950130,0.818413,0.000000,1.584953,3.351900,1.608835,2.156936,...,1.648293,1.266021,2.950130,0.818413,0.000000,1.584953,3.351900,1.608835,2.156936,1.140093
6,1.772084,2.620063,0.331978,3.484957,2.023159,1.584953,0.000000,3.613835,0.024273,2.426205,...,2.620063,0.331978,3.484957,2.023159,1.584953,0.000000,3.613835,0.024273,2.426205,0.692787
7,3.232857,1.924472,3.426276,0.654073,2.627480,3.351900,3.613835,0.000000,3.626976,1.240024,...,1.924472,3.426276,0.654073,2.627480,3.351900,3.613835,0.000000,3.626976,1.240024,3.020797
8,1.796228,2.641841,0.356196,3.501550,2.046917,1.608835,0.024273,3.626976,0.000000,2.441723,...,2.641841,0.356196,3.501550,2.046917,1.608835,0.024273,3.626976,0.000000,2.441723,0.714068
9,2.066745,1.072974,2.210810,1.073228,1.518779,2.156936,2.426205,1.240024,2.441723,0.000000,...,1.072974,2.210810,1.073228,1.518779,2.156936,2.426205,1.240024,2.441723,0.000000,1.798168


In [9]:
#creates data for the problem
def create_data_model(dist_matrix, num_vehicles, depot_index):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dist_matrix  # yapf: disable
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot_index
    return data

In [10]:
data = create_data_model(np.array(dist), 3, 4)

# Vehicle Routing Problem

In the Vehicle Routing Problem (VRP), the goal is to find optimal routes for multiple vehicles visiting a set of locations. (When there's only one vehicle, it reduces to the Traveling Salesperson Problem.)

But what do we mean by "optimal routes" for a VRP? One answer is the routes with the least total distance. However, if there are no other constraints, the optimal solution is to assign just one vehicle to visit all locations, and find the shortest route for that vehicle. This is essentially the same problem as the TSP.

A better way to define optimal routes is to minimize the length of the longest single route among all vehicles. This is the right definition if the goal is to complete all deliveries as soon as possible. The VRP example below finds optimal routes defined this way.

In later sections, we'll describe other ways of generalizing the TSP by adding constraints on the vehicles, including:

Capacity constraints: the vehicles need to pick up items at each location they visit, but have a maximum carrying capacity.
Time windows: each location must be visited within a specific time window.

## Create data

[(456, 320), # location 0 - the depot
(228, 0),    # location 1
(912, 0),    # location 2
(0, 80),     # location 3
(114, 80),   # location 4
(570, 160),  # location 5
(798, 160),  # location 6
(342, 240),  # location 7
(684, 240),  # location 8
(570, 400),  # location 9
(912, 400),  # location 10
(114, 480),  # location 11
(228, 480),  # location 12
(342, 560),  # location 13
(684, 560),  # location 14
(0, 640),    # location 15
(798, 640)]  # location 16

In [11]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                        data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

## Create the distance callback

To use the routing solver, you need to create a distance (or transit) callback: a function that takes any pair of locations and returns the distance between them. The easiest way to do this is using the distance matrix.

The following function creates the callback and registers it with the solver as transit_callback_index.

In [12]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [13]:
data['distance_matrix'][1][4]

0.8312117970442651

## Set the cost of travel

In this example, the arc cost evaluator is the transit_callback_index, which is the solver's internal reference to the distance callback. This means that the cost of travel between any two locations is just the distance between them. However, in general the costs can involve other factors as well.

You can also define multiple arc cost evaluators that depend on which vehicle is traveling between locations, using the method routing.SetArcCostEvaluatorOfVehicle(). For example, if the vehicles have different speeds, you could define the cost of travel between locations to be the distance divided by the vehicle's speed — in other words, the travel time.



In [14]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

## Add a distance dimension

To solve this VRP, you need to create a distance dimension, which computes the cumulative distance traveled by each vehicle along its route. You can then set a cost proportional to the maximum of the total distances along each route. Routing programs use dimensions to keep track of quantities that accumulate over a vehicle's route. See Dimensions for more details.

The following code creates the distance dimension, using the solver's AddDimension method. The argument transit_callback_index is the index for the distance_callback.

In [15]:
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    3000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

## Add solution printer

In [16]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

## Solve and print solution

In [17]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.time_limit.seconds = 150

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [18]:
print_solution(data, manager, routing, solution)

Objective: 305
Route for vehicle 0:
 4 ->  1 ->  21 ->  19 ->  17 ->  13 ->  10 ->  8 ->  6 ->  2 -> 4
Distance of the route: 2m

Route for vehicle 1:
 4 ->  11 ->  5 ->  0 -> 4
Distance of the route: 0m

Route for vehicle 2:
 4 ->  16 ->  15 ->  20 ->  9 ->  18 ->  14 ->  7 ->  3 ->  12 -> 4
Distance of the route: 3m

Maximum of the route distances: 3m


## Save routes to a list or array

As an alternative to printing the solution directly, you can save the route (or routes, for a VRP) to a list or array. This has the advantage of making the routes available in case you want to do something with them later. For example, you could run the program several times with different parameters and save the routes in the returned solutions to a file for comparison.

The following functions save the routes in the solution to any VRP (possibly with multiple vehicles) as a list (Python) or an array (C++).

In [19]:
def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

In [20]:
routes = get_routes(solution, routing, manager)
# Display the routes.
for i, route in enumerate(routes):
  print('Route', i, route)

Route 0 [4, 1, 21, 19, 17, 13, 10, 8, 6, 2, 4]
Route 1 [4, 11, 5, 0, 4]
Route 2 [4, 16, 15, 20, 9, 18, 14, 7, 3, 12, 4]


In [21]:
routes

[[4, 1, 21, 19, 17, 13, 10, 8, 6, 2, 4],
 [4, 11, 5, 0, 4],
 [4, 16, 15, 20, 9, 18, 14, 7, 3, 12, 4]]

# Plot results

In [26]:
jabee = jabee.to_crs('epsg:4326')

In [37]:
jabee['Hub_Flg'] = '0'
jabee.loc[4, 'Hub_Flg'] = '1'

In [40]:
px.set_mapbox_access_token(mapbox_token)

fig = px.scatter_mapbox(jabee,
                        lat=jabee.geometry.y,
                        lon=jabee.geometry.x,
                        hover_name="Company",
                        color = "Hub_Flg",
                        zoom=13,
                        marker_size = 10)
fig.show()

TypeError: scatter_mapbox() got an unexpected keyword argument 'marker_size'

In [47]:
jabee.geometry.y.mean()

14.582067554545459

In [65]:
jabee['color'] = jabee.Hub_Flg.map(dict(zip(['0', '1'], px.colors.qualitative.Plotly)))
jabee['color']

0     #636EFA
1     #636EFA
2     #636EFA
3     #636EFA
4     #EF553B
5     #636EFA
6     #636EFA
7     #636EFA
8     #636EFA
9     #636EFA
10    #636EFA
11    #636EFA
12    #636EFA
13    #636EFA
14    #636EFA
15    #636EFA
16    #636EFA
17    #636EFA
18    #636EFA
19    #636EFA
20    #636EFA
21    #636EFA
Name: color, dtype: object

In [69]:
fig = go.Figure(go.Scattermapbox(
        lat=jabee.geometry.y,
        lon=jabee.geometry.x,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,
            color = 'green',#jabee['color'] ,
        ),
        name = 'Spoke'
    ))


fig.add_trace(go.Scattermapbox(
        lat=jabee.loc[jabee.Hub_Flg == '1'].geometry.y,
        lon=jabee.loc[jabee.Hub_Flg == '1'].geometry.x,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,
            color = jabee.loc[jabee.Hub_Flg == '1']['color'] ,
        ),
        name = 'Hub'
    ))

fig.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        # bearing=0,
        center=go.layout.mapbox.Center(
            lat=jabee.geometry.y.mean(),
            lon=jabee.geometry.x.mean(),
        ),
        # pitch=0,
        zoom=12
    )
)

fig.show()

In [75]:
colors = px.colors.qualitative.Plotly

In [84]:
jabee.loc[routes[0]].geometry.y

4     14.575531
1     14.577854
21    14.583490
19    14.585268
17    14.585119
13    14.583228
10    14.583490
8     14.585268
6     14.585119
2     14.583228
4     14.575531
dtype: float64

In [86]:
fig = go.Figure()

for idx, route in enumerate(routes):
    fig.add_trace(go.Scattermapbox(
            mode = "markers+lines",
            lat = jabee.loc[route].geometry.y,
            lon = jabee.loc[route].geometry.x,
            marker=go.scattermapbox.Marker(
                    size=14,
                    color = colors[idx],
                ),
                name = f'Route {idx}'
            # marker = {'size': 15, 'color': colors[idx]}
            ))

fig.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        # bearing=0,
        center=go.layout.mapbox.Center(
            lat=jabee.geometry.y.mean(),
            lon=jabee.geometry.x.mean(),
        ),
        # pitch=0,
        zoom=12
    )
)

fig.show()

In [111]:
def compute_arrow_coords(A, B):
    
    #Workaround to get the arrow at the end of an edge AB
    #A - coords of point 1, B - coords of point B
    l = 0.1  # the arrow length
    widh =0.035  #2*widh is the width of the arrow base as triangle

    # A = np.array([locations['lon'][5], locations['lat'][5]])
    # B = np.array([locations['lon'][0], locations['lat'][0]])
    v = B-A
    w = v/np.linalg.norm(v)     
    u  =np.array([-v[1], v[0]])  #u orthogonal on  w
            
    P = B-l*w
    S = P - widh*u
    T = P + widh*u

    return S, T

In [108]:
S, T

(array([121.9877439187512, 15.136955222010615], dtype=object),
 array([121.9882827367512, 15.136046377010613], dtype=object))

In [114]:
A, B

(array([121.0547872, 14.5832282], dtype=object),
 array([121.0418037, 14.575530800000005], dtype=object))

In [112]:
fig = go.Figure()

# for idx, route in enumerate(routes):
#     fig.add_trace(go.Scattermapbox(
#             mode = "markers+lines",
#             lat = jabee.loc[route].geometry.y,
#             lon = jabee.loc[route].geometry.x,
#             marker=go.scattermapbox.Marker(
#                     size=14,
#                     color = colors[idx],
#                 ),
#                 name = f'Route {idx}'
#             # marker = {'size': 15, 'color': colors[idx]}
#             ))

for i in range(len(routes[0])-1):
    A = np.array(jabee.loc[routes[0][i], ['Longitude', 'Latitude']])
    B = np.array(jabee.loc[routes[0][i+1], ['Longitude', 'Latitude']])
    S, T = compute_arrow_coords(A, B)
    fig.add_trace(go.Scattergeo(lon = [S[0], T[0], B[0], S[0]], 
                            lat =[S[1], T[1], B[1], S[1]], 
                            mode='lines', 
                            fill='toself', 
                            fillcolor='blue', 
                            line_color='blue'))

fig.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        # bearing=0,
        center=go.layout.mapbox.Center(
            lat=jabee.geometry.y.mean(),
            lon=jabee.geometry.x.mean(),
        ),
        # pitch=0,
        zoom=12
    )
)

fig.show()

In [99]:
compute_arrow_coords(np.array(jabee.iloc[0][['Longitude', 'Latitude']]), 
                        np.array(jabee.iloc[1][['Longitude', 'Latitude']]))

(array([122.06243259674207, 14.185637080053766], dtype=object),
 array([122.06210466074208, 14.184778887053767], dtype=object))

In [91]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

np.random.seed(1)
I_list = list(np.random.randint(0,3229,10))
skiplist = set(range(1, 3229)) - set(I_list)
locations = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv', skiprows=skiplist)

scale = 5000
fig = go.Figure()
fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = locations['lon'],
        lat = locations['lat'],
        text = locations['name'],
        textfont = {"color": 'black',
                    "family":'Times New Roman',
                    "size":14},
        textposition="top center",
        name = "Candidate Facility",
        mode ="markers+text",
        marker = dict(
            size = 10,
            color = "black",
            line_color='black',
            line_width=0.5,
            sizemode = 'area')))

fig.add_trace(go.Scattergeo(
    lat = [locations['lat'][5], locations['lat'][0]], 
    lon = [locations['lon'][5], locations['lon'][0]],
    mode = 'lines',
    line = dict(width = 1.5, color = 'blue'),
))

#Workaround to get the arrow at the end of an edge AB

l = 1.1  # the arrow length
widh =0.035  #2*widh is the width of the arrow base as triangle

A = np.array([locations['lon'][5], locations['lat'][5]])
B = np.array([locations['lon'][0], locations['lat'][0]])
v = B-A
w = v/np.linalg.norm(v)     
u  =np.array([-v[1], v[0]])  #u orthogonal on  w
         
P = B-l*w
S = P - widh*u
T = P + widh*u

fig.add_trace(go.Scattergeo(lon = [S[0], T[0], B[0], S[0]], 
                            lat =[S[1], T[1], B[1], S[1]], 
                            mode='lines', 
                            # fill='toself', 
                            fillcolor='blue', 
                            line_color='blue'))
#------Display your text at the middle of the segment AB
fig.add_trace(go.Scattergeo(lon =[0.5*(A+B)[0]], lat = [0.5*(A+B)[1]], mode='text', text='               Your text'))

fig.update_layout(width=900, height=750,
        showlegend = False,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)'))
fig.show()

# VRP with capacity constraints

Next, we describe an example of a VRP with capacity constraints. The example extends the previous VRP example and adds the following requirements. At each location there is a demand corresponding to the quantity of the item to be picked up. Also, each vehicle has a maximum capacity of 15. (We aren't specifying units for the demands or capacity.)

The grid below shows the locations to visit in blue and the company location in black. The demands are shown at the lower right of each location. See Location coordinates in the VRP section for more details about how the locations are defined.

In [7]:
data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
data['vehicle_capacities'] = [15, 15, 15, 15]

## Add the demand callback

Unlike the distance callback, which takes a pair of locations as inputs, the demand callback only depends on the location (from_node) of the delivery.

In [8]:
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

True

## Add the solution printer

In [9]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [10]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [12]:
print_solution(data, manager, routing, solution)

Objective: 6208
Route for vehicle 0:
 0 Load(0) ->  4 Load(4) ->  3 Load(6) ->  1 Load(7) ->  7 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 1:
 0 Load(0) ->  14 Load(4) ->  16 Load(12) ->  10 Load(14) ->  9 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 2:
 0 Load(0) ->  12 Load(2) ->  11 Load(3) ->  15 Load(11) ->  13 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 3:
 0 Load(0) ->  8 Load(8) ->  2 Load(9) ->  6 Load(13) ->  5 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Total distance of all routes: 6208m
Total load of all routes: 60


What happens if a problem has no solution ?
A routing problem with constraints, such as a CVRP, might not have a feasible solution — for example, if the total quantity of the items being transported exceeds the total capacity of the vehicles. If you try to solve such a problem, the solver might run an exhaustive search which takes so long that eventually you have to give up and interrupt the program.

Usually this won't be an issue. But here are a couple of ways to prevent your program from running a long time when a problem has no solution:

Set a time limit in the program, which stops the search even if no solution has been found. However, keep in mind that if the problem has a solution that requires a lengthy search, the program might reach the time limit before finding the solution.
Set penalties for dropping visits to locations. This allows the solver to return a "solution" that doesn't visit all locations in case the problem is infeasible. See Penalties and Dropping Visits.
In general, it can be hard to tell if a given problem has a solution. Even for a CVRP in which total demand doesn't exceed total capacity, determining whether all the items will fit in the vehicles is a version of the multiple knapsack problem.



# Setting start and end locations for routes

So far, we have assumed that all vehicles start and end at a single location, the depot. You can also set possibly different start and end locations for each vehicle in the problem. To do so, pass two vectors, containing the indices of the start and end locations, as inputs to the RoutingModel method in the main function. Here's how to create the start and end vectors in the data section of the program:

In [14]:
data['starts'] = [1, 2, 15, 16]
data['ends'] = [0, 0, 0, 0]

In [15]:
"""Simple Vehicles Routing Problem."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    data['num_vehicles'] = 4
    data['starts'] = [1, 2, 15, 16]
    data['ends'] = [0, 0, 0, 0]
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['starts'],
                                           data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        2000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 115794
Route for vehicle 0:
 1 ->  4 ->  3 ->  7 -> 0
Distance of the route: 1004m

Route for vehicle 1:
 2 ->  6 ->  8 ->  5 -> 0
Distance of the route: 936m

Route for vehicle 2:
 15 ->  11 ->  12 ->  13 -> 0
Distance of the route: 936m

Route for vehicle 3:
 16 ->  14 ->  10 ->  9 -> 0
Distance of the route: 1118m

Maximum of the route distances: 1118m


# VRP with various Pickups and Deliveries

In this section we describe a VRP in which each vehicle picks up items at various locations and drops them off at others. The problem is to assign routes for the vehicles to pick up and deliver all the items, while minimizing the length of the longest route.